In [ ]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import scipy.sparse as sp
import numpy as np
import utils_graphsage_1 as utils
import torch
import torch
from collections import defaultdict
import numpy as np
import time
import json
import random
import pandas as pd
import os

In [ ]:
#toeplitz(range(4))

In [ ]:
#seed=114514
seed = 41222
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.seed_all()

# **Read data**

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [ ]:
data= pd.read_csv("../data/opsahl-ucsocial/data.csv")
data = data.drop_duplicates(subset=['start','end'])
node_to_id = dict()
node_type_dict = dict()
for node in list(data['start']) + list(data['end']):
    node = int(node)
    if node not in node_to_id:
        node_to_id[node] = len(node_to_id)
for node in list(data['start']):
    node = node_to_id[int(node)]
    node_type_dict[node] = 1 ### user type node
for node in list(data['end']):
    node = node_to_id[int(node)]
    node_type_dict[node] = 0 ### item type node

#edges = defaultdict(lambda: 0)
train_ones = []
feature_dict = {}
for start,end in data[['start','end']].values:
    start,end = int(start),int(end)
    start = node_to_id[start]
    end = node_to_id[end]
    train_ones.append([int(start),int(end)])
train_ones= np.array(train_ones)
print(len(train_ones))
print(train_ones[:5])
### There can be multiple edges between same node  pairs


adj_sparse = np.zeros((np.max(train_ones)+1,np.max(train_ones)+1))
for e in train_ones:
    adj_sparse[e[0],e[1]]=1
    adj_sparse[e[1],e[0]]=1
    
adj_sparse = sp.coo_matrix(adj_sparse).tocsr()

# lcc = utils.largest_connected_components(adj_sparse)
# adj_sparse= adj_sparse[lcc,:][:,lcc]
_N = adj_sparse.shape[0]
print('n',_N)
_Edges=[]
for x in np.column_stack(adj_sparse.nonzero()):
    if not x[0]==x[1]:
        _Edges.append((x[0],x[1]))
_num_of_edges=int(len(_Edges)/2)
print('m',_num_of_edges)

dic=defaultdict(set)
for x in _Edges:
    a1=x[0]
    a2=x[1]
    dic[a1].add(a2)
    dic[a2].add(a1)
    

adj_origin=np.zeros((_N,_N)).astype(int)  ### extra dimension for node type
for (i,j) in _Edges:
    adj_origin[i][j]=1
    adj_origin[j][i]=1
assert(np.sum(adj_origin==adj_origin.T)==_N*_N)
assert(np.sum(adj_origin)==_num_of_edges*2)


embedding_dim=128

graphsagemodel=utils.GraphSAGE(_N=_N,_M=_num_of_edges,adj_origin=adj_origin,feat_matrix = adj_origin,
                                         adj_dic=dic,embedding_dim=embedding_dim)


# **1.Get link prediction model and embedding**

# *1.1 Load pretrained model*

In [ ]:
graphsagemodel.load_model(path='models/gf_msg/graphsage0_87.pth',embedding_path='models/gf_msg/embedding_matrix0_87.pth.npy')
embedding_matrix_numpy=graphsagemodel.embedding_matrix_numpy
link_prediction_model=graphsagemodel.graphsage_link_prediction_from_embedding_one_to_other
predict_adj=utils.evaluate_overlap_torch(_N=_N,
                                                    _num_of_edges=_num_of_edges,
                                                    adj_origin=adj_origin,
                                                    embedding_matrix_numpy=embedding_matrix_numpy,
                                                    link_prediction_from_embedding_one_to_other=link_prediction_model)

In [ ]:
import pickle
id_to_node = {value:key for key,value in node_to_id.items()}
node_embeddings = {}
for node in node_to_id:
    _id = node_to_id[node]
    node_embeddings[node] = embedding_matrix_numpy[_id]
pickle.dump(node_embeddings,open("models/gf_msg/embeddings.pkl","wb"))
len(node_to_id),embedding_matrix_numpy.shape

In [ ]:
metric_embedding=utils.compute_graph_statistics(predict_adj)
metric_origin=utils.compute_graph_statistics(adj_origin)

In [ ]:
for x in metric_origin:
    print('%-25s origin:%17.8f, link_pred:%17.8f'%(x,metric_origin[x],metric_embedding[x]))

In [ ]:
def compute_different():
    different_idx=[]
    visited={}
    for i in range(len(embedding_matrix_numpy)):
        if i not in visited:
            different_idx.append(i)
            visited[i]=True
            for j in range(i+1,len(embedding_matrix_numpy)):
                if np.linalg.norm(embedding_matrix_numpy[i]-embedding_matrix_numpy[j])<1e-5:
                    if j not in visited:
                        visited[j]=True
        if i%100==0:
            print('\r%d/%d'%(i,_N),end="")
    return different_idx
different_idx=compute_different()
embeddings_training=embedding_matrix_numpy[different_idx,:]
print(embeddings_training.shape)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(2, n_iter=15000,verbose=1,perplexity = 30) #perplexity=40,
projected = tsne.fit_transform(embeddings_training)
x = []
y = []
for value in projected:
    x.append(value[0])
    y.append(value[1])

FS = (10, 8)
fig, ax = plt.subplots(figsize=FS)
# Make points translucent so we can visually identify regions with a high density of overlapping points
ax.scatter(x, y, alpha=.1);

# **2.GAN generate new embeddings**

In [ ]:
batch_size=256
noise_dim=16
g_hidden_dim=[32,64,100]
d_hidden_dim=[100,64,32]
lendataloader=20
Diter=4
Giter=1
epoch_numbers=30000
eval_epoch=400


In [ ]:
#embeddings_training = embeddings_training / np.linalg.norm(embeddings_training, axis=1)[:, np.newaxis]


# **2.1 GAN training**

In [ ]:
import time
save_idx=0
global_best_mmd=10000
while(1):
    start_time=time.time()
    print(save_idx)
    flag,best_mmd=utils.gan_train(embeddings_training,batch_size=batch_size,noise_dim=noise_dim,
                                   g_hidden_dim=g_hidden_dim,d_hidden_dim=d_hidden_dim,
                                   lendataloader=lendataloader,Diter=Diter,Giter=Giter,epoch_numbers=epoch_numbers,eval_epoch=eval_epoch,
                                   save_idx=save_idx,learning_rate=1e-4,
                                   mmd_beta=0.1,mmd_criterion=0.003,mmd_best_criterion=0.001,dirs='models/gf_msg/')
    global_best_mmd=min(global_best_mmd,best_mmd)
    if flag==True:
        break
    save_idx=save_idx+1
    print('Using time:%.2f'%(time.time()-start_time))
    print(best_mmd)
    print(global_best_mmd)

In [ ]:
embeddings_training[100]

# **2.2 Load pretrained/trained gan model**

# *2.2.1 Load provided model*

In [ ]:
netG = utils.Generator(noise_dim=noise_dim,embedding_dim=embedding_dim, g_hidden_dim=g_hidden_dim,batch_size=batch_size).cuda()

In [ ]:
netG.load_state_dict(torch.load('models/gf_msg/bestG.pth'))

# *2.2.2compute ECDF*

In [ ]:
utils.eval_plot(netG,embedding_matrix=embedding_matrix_numpy,noise_dim=16,mmd_beta=0.1)

In [ ]:
'j'

# **3.Sample**

# *3.1Generate embeddings*

In [ ]:
noise= torch.randn(_N, noise_dim).cuda()
generate_data=netG(noise)
generate_data=generate_data.detach().to('cpu').numpy()
print(generate_data.shape)
np.save('models/gf_msg/gan_embeddings_N.npy', generate_data)



In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(2, n_iter=15000,verbose=1,perplexity = 30) #perplexity=40,
projected = tsne.fit_transform(generate_data)
x = []
y = []
for value in projected:
    x.append(value[0])
    y.append(value[1])

#     for i in range(len(x)):
#         plt.scatter(x[i],y[i])
#         plt.annotate(labels[i],
#                      xy=(x[i], y[i]),
#                      xytext=(5, 2),
#                      textcoords='offset points',
#                      ha='right',
#                      va='bottom')
FS = (10, 8)
fig, ax = plt.subplots(figsize=FS)
# Make points translucent so we can visually identify regions with a high density of overlapping points
ax.scatter(x, y, alpha=.1);

In [ ]:
probability_matrix_generate=utils.generate_probability_matrix(_N,generate_data,
                                                                        link_prediction_model)
_,graphic_seq_generate=utils.evaluate_overlap_torch_generate(_N,_num_of_edges,
                                                                                  probability_matrix_generate)

In [ ]:
generate_graph=utils.revised_Havel_Hakimmi_Algorithm(_N,_num_of_edges,dic,probability_matrix_generate,graphic_seq_generate)

In [ ]:
tp=0
tn=0
fp=0
fn=0
for i in range(_N):
    for j in range(_N):
        if generate_graph[i,j]==1 and adj_origin[i,j]==1:
            tp=tp+1
        if generate_graph[i,j]==0 and adj_origin[i,j]==1:
            fp=fp+1
        if generate_graph[i,j]==1 and adj_origin[i,j]==0:
            fn=fn+1
        if generate_graph[i,j]==0 and adj_origin[i,j]==0:
            tn=tn+1
    print('\r%d/%d'%(i,_N),end="")
print('\n')
print('Edge overlap between generate graph and original graph')
print(generate_graph.shape)
total_num=_N*_N
print('True Positve:%d, %.2f'%(tp,tp/(tp+fp)))
print('False Positve:%d, %.2f'%(fp,fp/(tp+fp)))
print('True Negative:%d, %.2f'%(tn,tn/(tn+fn)))
print('False Negative:%d, %.2f'%(fn,fn/(tn+fn)))
print('Positive:%.2f'%((tp+fp)/total_num))
print('Negative:%.2f'%((tn+fn)/total_num))

In [ ]:
metric_graphic_sq_generate=utils.compute_graph_statistics(generate_graph)
metric_graphic_sq_generate['edge_overlap']=tp/(tp+fp)

In [ ]:
for x in metric_origin:
    print('%-22s origin:%17.8f, link_pred:%17.8f, generate:%17.8f'%(x,metric_origin[x],
                                                                    metric_embedding[x],metric_graphic_sq_generate[x]))